### Import des bibliothéques
---

In [2]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score,confusion_matrix, classification_report, roc_curve, auc

###  Chargement des données
---

In [3]:
df = pd.read_csv("../data/conversion_data_train.csv", encoding="latin-1")
df.head()

,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0


### Modéle LogiticRegression avec stratify (déséquilibre target)
---

In [12]:
# Liste des features et de la variable cible
features_list = ['total_pages_visited', 'age', 'new_user', 'source', 'country']
target_variable = 'converted'

# Séparer les variables indépendantes (X) et la variable cible (Y)
X = df[features_list]
Y = df[target_variable]

# Diviser le jeu de données en ensembles d'entraînement (80%) et de test (20%), stratify=repartition des classe(0,1) reste proportionnelle
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Pipeline pour les colonnes numériques : Imputation + Standardisation
numerical_columns = ['total_pages_visited','age', 'new_user' ]
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Remplacer les valeurs manquantes par la moyenne
    ('standardization', StandardScaler())  # Standardisation des données numériques
])

# Pipeline pour les colonnes catégorielles : Imputation + Encodage One-Hot
categorical_columns = ['country', 'source']
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Remplacer les valeurs manquantes par la modalité la plus fréquente
    ('encoder', OneHotEncoder(drop='first'))  # Encodage One-Hot des variables catégorielles
])

# Appliquer les pipelines numériques et catégoriels sur les colonnes spécifiées
feature_encoder = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_columns),
    ('cat', categorical_pipeline, categorical_columns)
])

# Appliquer les transformations sur les données d'entraînement et de test
X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test)

# Initialisation du modèle de régression logistique
lr = LogisticRegression()

# Entraînement du modèle sur les données d'entraînement
lr.fit(X_train, Y_train)

# Prédictions
Y_train_pred = lr.predict(X_train)
Y_test_pred = lr.predict(X_test)

# Métriques 
print("--- Évaluation des performances du modèle --- \n")

# Train
print(" --- Train ---\n")
print(f"Accuracy  : {accuracy_score(Y_train, Y_train_pred):.4f}")
print(f"Precision : {precision_score(Y_train, Y_train_pred):.4f}")
print(f"Recall    : {recall_score(Y_train, Y_train_pred):.4f}")
print(f"F1 Score  : {f1_score(Y_train, Y_train_pred):.4f}\n")

# Test
print("--- Test ---\n ")
print(f"Accuracy  : {accuracy_score(Y_test, Y_test_pred):.4f}")
print(f"Precision : {precision_score(Y_test, Y_test_pred):.4f}")
print(f"Recall    : {recall_score(Y_test, Y_test_pred):.4f}")
print(f"F1 Score  : {f1_score(Y_test, Y_test_pred):.4f}")



--- Évaluation des performances du modèle --- 

 --- Train ---

Accuracy  : 0.9862
Precision : 0.8549
Recall    : 0.6874
F1 Score  : 0.7620

--- Test ---
 
Accuracy  : 0.9865
Precision : 0.8653
Recall    : 0.6895
F1 Score  : 0.7675


### Sauvegarde des résultats
---

In [14]:

new_rows = [{"model": "lr", "f1": f1_score(Y_test, Y_test_pred), "recall": recall_score(Y_test, Y_test_pred), "accuracy":accuracy_score(Y_test, Y_test_pred)}]

scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../data/scores_models.csv", mode="a", header=False, index=False)